In [ ]:
# Imports
# Standard Libraries of Python
import itertools
from collections import Counter
from typing import Tuple, Union
from decimal import Decimal, ROUND_HALF_UP, getcontext
from datetime import datetime
getcontext().prec = 5
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# Dependencies
import pandas as pd
import numpy as np
from pandas import DataFrame, Index

# Libraries made for this Proyect
from backend.src.parse import Criteria
from backend.src import pick_numbers
file_route = 'database/files/'

In [ ]:
# Test of Tomorrow Numbers
euromillions = Criteria()

euromillions.define_odd_even()
euromillions.define_low_high()
euromillions.define_combinations_skips()
euromillions.groups_info()
euromillions.apply_transformation()
euromillions.count_skips()
euromillions.skips_for_last_12_draws()
euromillions.get_natural_rotations()
euromillions.get_numbers_clasification()
euromillions.draw_skips()
euromillions.skips_evaluation()

euromillions.year_criterion()
euromillions.rotation_criterion()
euromillions.position_criterion()
euromillions.group_criterion()
euromillions.numbers_of_tomorrow()

lotto = pick_numbers.Selection(euromillions)
lotto.first_number()
lotto.suggested_numbers()

content = itertools.combinations(lotto._selected_numbers,5)

tickets = []
for combination in content:
    tickets.append(combination)
    print(combination)

combinations = [(1, 4), (2, 3), (3, 2), (4, 1), (5, 0), (0, 5)]
# Crear variables predictoras (X) y la variable objetivo (y)
X = euromillions.combinations_skips[['low', 'high'] + [f'days_since_{comb[0]}_{comb[1]}' for comb in combinations]]  # Incluimos las columnas de días
y = euromillions.combinations_skips['odd']  # Variable objetivo (puedes cambiar a otra variable si lo prefieres)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluación
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

last_draw = euromillions.combinations_skips.iloc[-1]

# Crear las características de entrada para la predicción
last_features = [
    last_draw['low'],              # Característica low
    last_draw['high'],             # Característica high
    last_draw['days_since_1_4'],   # Días desde que salió la combinación (1, 4)
    last_draw['days_since_2_3'],   # Días desde que salió la combinación (2, 3)
    last_draw['days_since_3_2'],   # Días desde que salió la combinación (3, 2)
    last_draw['days_since_4_1'],   # Días desde que salió la combinación (4, 1)
    last_draw['days_since_5_0'],   # Días desde que salió la combinación (5, 0)
    last_draw['days_since_0_5']    # Días desde que salió la combinación (0, 5)
]

# Convertir las características en un DataFrame o un numpy array
last_features = np.array(last_features).reshape(1, -1)  # Reshape para tener un solo conjunto de datos

# Realizar la predicción
y_pred_next = model.predict(last_features)
print(f'For the next draw, there is a chance that the amount of odd numbers will be {y_pred_next}')

In [ ]:
euromillions.evaluation.iloc[-20:].mean()

In [ ]:
euromillions.df.iloc[-10:]

In [ ]:
picking_low = []
for n, ticket in enumerate(tickets):
    value = filter(lambda x: x <= 25, ticket)
    amount = sum(1 for x in list(value) if x > 0)
    if amount == 2:
        picking_low.append(n)

picking_odd = []
for n, ticket in enumerate(tickets):
    value = filter(lambda x: x % 2 != 0, ticket)
    amount = sum(1 for x in list(value) if x > 0)
    if amount == 3:
        picking_odd.append(n)

In [ ]:
euromillions.df.drop(1803)

In [ ]:
euromillions.future_groups

In [ ]:
buy = []
for ticket in picking_low:
    if ticket in picking_odd:
        buy.append(tickets[ticket])

In [ ]:
buy

In [ ]:
euromillions.recommended_numbers

In [ ]:
df = pd.DataFrame(buy,columns=['1','2','3','4','5'])
df = df.iloc[:,0:6].apply(lambda x: pd.Series(sorted(x)),axis=1)
df.to_csv(f'{file_route}combinations.csv',index=False)

In [ ]:
combinations = [(1, 4), (2, 3), (3, 2), (4, 1), (5, 0), (0, 5)]
# Crear variables predictoras (X) y la variable objetivo (y)
X = euromillions.combinations_skips[['low', 'high'] + [f'days_since_{comb[0]}_{comb[1]}' for comb in combinations]]  # Incluimos las columnas de días
y = euromillions.combinations_skips['odd']  # Variable objetivo (puedes cambiar a otra variable si lo prefieres)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluación
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}. Value for odd number: {y_pred}')

In [ ]:
count = 0
fails = 0
for draw in range(0,len(euromillions.df)-1):
    # Obtener el último sorteo
    last_draw = euromillions.combinations_skips.iloc[draw]

    # Crear las características de entrada para la predicción
    last_features = [
        last_draw['low'],              # Característica low
        last_draw['high'],             # Característica high
        last_draw['days_since_1_4'],   # Días desde que salió la combinación (1, 4)
        last_draw['days_since_2_3'],   # Días desde que salió la combinación (2, 3)
        last_draw['days_since_3_2'],   # Días desde que salió la combinación (3, 2)
        last_draw['days_since_4_1'],   # Días desde que salió la combinación (4, 1)
        last_draw['days_since_5_0'],   # Días desde que salió la combinación (5, 0)
        last_draw['days_since_0_5']    # Días desde que salió la combinación (0, 5)
    ]

    # Convertir las características en un DataFrame o un numpy array
    last_features = np.array(last_features).reshape(1, -1)  # Reshape para tener un solo conjunto de datos

    # Realizar la predicción
    y_pred_next = model.predict(last_features)

    # Imprimir el resultado
    next = draw + 1
    v = euromillions.combinations_skips.iloc[next,3]
    if y_pred_next == v:
        print(f'acerto el modelo con la prediccion {y_pred_next}, donde el valor {v}, se cumplio para el sorteo {next}')
        count += 1
    else:
        fails += 1
        print(f'fallo la prediccion {y_pred_next}, donde el valor {v}, no corresponde con {next}')

print(f'correctos: {count}')
print(f'Fallos: {fails}')

In [ ]:
n_predictions = len(y_test)
random_predictions = np.random.choice([0, 1, 2, 3, 4, 5], size=n_predictions, replace=True)
random_accuracy = accuracy_score(y_test, random_predictions)
random_accuracy

In [ ]:
# Test of Tomorrow Stars

euromillions = Criteria(is_star=True)

euromillions.apply_transformation(is_star=True)
euromillions.count_skips(is_star=True)
euromillions.get_natural_rotations(is_star=True)
print(euromillions.year_history)

print(euromillions.aprox_rotation)
print(euromillions.exact_rotation)

In [ ]:
euromillions.counts.max()

In [ ]:
euromillions.df.loc[euromillions.df['dates'] == datetime(2016,9,27)]

In [ ]:
def find_group(value: int) -> str | None:
    stars_group = {
    'A': [1,3,5],
    'B': [2,4,6],
    'C': [7,9,11],
    'D': [8,10,12]
}
    
    for k, v in stars_group.items():
        if value in v:
            return k
    
    return None

euromillions.df_stars['group_0'] = euromillions.df_stars['star_1'].apply(find_group) 
euromillions.df_stars['group_1'] = euromillions.df_stars['star_2'].apply(find_group)

In [ ]:
euromillions.df

In [ ]:
euromillions.df_stars.iloc[-20:]

In [ ]:
euromillions.df_stars['combined_groups'] = euromillions.df_stars[['group_0','group_1']].apply(lambda row: [value for value in row if isinstance(value, str)], axis = 1)

In [ ]:
euromillions.df.loc[:,'nro1']

In [ ]:
star = euromillions.df_stars.iloc[:,[1,2,3,4,5,6]]
# Definir las combinaciones posibles (low, high)
combinations = [('B','D'), ('B','C'), ('A','C'), ('A','D'), ('A','B'), ('C','D'), ('B','A'), ('A','A'), ('B','B'), ('D','C'), ('C','C'), ('D','D')]

# Inicializamos las nuevas columnas para almacenar los días desde la última aparición
for comb in combinations:
    col_name = f'days_since_{comb[0]}_{comb[1]}'
    star[col_name] = None

# Inicializamos el contador de días
days_since = {comb: 0 for comb in combinations}

# Iteramos sobre las filas para calcular los días desde la última aparición de cada combinación
for i in range(863,len(euromillions.df)):
    for comb in combinations:
        col_name = f'days_since_{comb[0]}_{comb[1]}'
        
        if star.loc[i, 'combined_groups'] == list(comb):
            # Si la combinación actual coincide, reiniciamos el contador a 0
            days_since[comb] = 0
        else:
            # Si no coincide, incrementamos el contador de días
            days_since[comb] += 1
        
        # Asignamos el valor del contador de días en la nueva columna
        star.loc[i, col_name] = days_since[comb]

In [ ]:
avg = star.iloc[:,6:].mean().to_frame().T
last_draw = star.iloc[-1]

last_features = [
    last_draw['star_1'],
    last_draw['star_2'],
    last_draw['group_1'],              # Característica low
    last_draw['days_since_B_D'],             # Característica high
    last_draw['days_since_B_C'],   # Días desde que salió la combinación (1, 4)
    last_draw['days_since_A_C'],   # Días desde que salió la combinación (2, 3)
    last_draw['days_since_A_D'],   # Días desde que salió la combinación (3, 2)
    last_draw['days_since_A_B'],   # Días desde que salió la combinación (4, 1)
    last_draw['days_since_C_D'],   # Días desde que salió la combinación (5, 0)
    last_draw['days_since_B_A'],
    last_draw['days_since_A_A'],
    last_draw['days_since_B_B'], # Días desde que salió la combinación (0, 5)
    last_draw['days_since_D_C'],
    last_draw['days_since_C_C'],
    last_draw['days_since_D_D']
]

last_features = np.array(last_features).reshape(1, -1)
last_draw = last_draw.to_frame().T

In [ ]:
avg

In [ ]:
for i in avg.columns.to_list():
    if last_draw[i].values == 0:
        print('descartado, acaba de salir')
    elif avg[i].values < last_draw[i].values:
        print(f'{i} por encima de la media, escoger esta serie')
    elif avg[i].values > last_draw[i].values and abs(avg[i].values - last_draw[i].values):
        print('preferencia')
    # elif last_draw[i].values > avg[i].values

In [ ]:
last_draw

In [ ]:
last_draw = star.iloc[-1:]

In [ ]:
star.iloc[-20:]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Crear el DataFrame (sustituye esto por la carga de tus datos reales)
data = star.copy()  # Aquí carga el DataFrame real.

# Separar las características y la variable objetivo
X = data.drop(columns=['draw','star_1','star_2','group_1','combined_groups'])
y = data['group_0']

# Convertir las variables categóricas a numéricas si es necesario
categorical_columns = ["group_0"]
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Codificar el objetivo (combined_groups)
le_target = LabelEncoder()
y = le_target.fit_transform(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
X_test

In [ ]:
categorical_columns = ['group_0', 'group_1']
for col in categorical_columns:
    le = LabelEncoder()
    star[col] = le.fit_transform(star[col])

In [ ]:
last_draw = star.iloc[-1]

last_features = [
    last_draw['star_1'],
    last_draw['star_2'],
    last_draw['group_1'],              # Característica low
    last_draw['days_since_B_D'],             # Característica high
    last_draw['days_since_B_C'],   # Días desde que salió la combinación (1, 4)
    last_draw['days_since_A_C'],   # Días desde que salió la combinación (2, 3)
    last_draw['days_since_A_D'],   # Días desde que salió la combinación (3, 2)
    last_draw['days_since_A_B'],   # Días desde que salió la combinación (4, 1)
    last_draw['days_since_C_D'],   # Días desde que salió la combinación (5, 0)
    last_draw['days_since_B_A'],
    last_draw['days_since_A_A'],
    last_draw['days_since_B_B'], # Días desde que salió la combinación (0, 5)
    last_draw['days_since_D_C'],
    last_draw['days_since_C_C'],
    last_draw['days_since_D_D']
]

last_features = np.array(last_features).reshape(1, -1)

In [ ]:
last_features

In [ ]:
y_pred_next = model.predict(last_features)

In [ ]:
star.iloc[906]

In [ ]:
count = 0
fails = 0
for draw in range(863,len(euromillions.df)-1):
    # Obtener el último sorteo
    last_draw = star.loc[draw]

    # Crear las características de entrada para la predicción
    last_features = [
        last_draw['group_1'],              # Característica low
        last_draw['days_since_B_D'],             # Característica high
        last_draw['days_since_B_C'],   # Días desde que salió la combinación (1, 4)
        last_draw['days_since_A_C'],   # Días desde que salió la combinación (2, 3)
        last_draw['days_since_A_D'],   # Días desde que salió la combinación (3, 2)
        last_draw['days_since_A_B'],   # Días desde que salió la combinación (4, 1)
        last_draw['days_since_C_D'],   # Días desde que salió la combinación (5, 0)
        last_draw['days_since_B_A'],
        last_draw['days_since_A_A'],
        last_draw['days_since_B_B'], # Días desde que salió la combinación (0, 5)
        last_draw['days_since_D_C'],
        last_draw['days_since_C_C'],
        last_draw['days_since_D_D']
    ]

    # Convertir las características en un DataFrame o un numpy array
    last_features = np.array(last_features).reshape(1, -1)  # Reshape para tener un solo conjunto de datos

    # Realizar la predicción
    y_pred_next = model.predict(last_features)

    # Imprimir el resultado
    next = draw + 1
    v = star.loc[next,'group_0']
    if y_pred_next == v:
        print(f'acerto el modelo con la prediccion {y_pred_next}, donde el valor {v}, se cumplio para el sorteo {next}')
        count += 1
    else:
        fails += 1
        print(f'fallo la prediccion {y_pred_next}, donde el valor {v}, no corresponde con {next}')

print(f'correctos: {count}')
print(f'Fallos: {fails}')

In [ ]:
euromillions.df_stars